<a href="https://colab.research.google.com/github/dhu16/alpaca-trading-bot/blob/main/alpaca.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Dual moving average
!pip install alpaca_trade_api
import alpaca_trade_api as tradeapi
import threading
import pandas as pd
from datetime import date
from datetime import timedelta
from decimal import Decimal

     |████████████████████████████████| 204kB 7.1MB/s 
     |████████████████████████████████| 81kB 6.0MB/s 


In [3]:
#Auth
api_key = 'PKQW66RE66GPOJCPVNDY'
api_secret = 'o9HTNZ6JTwCTz3lmwrWUq08xK1klUjtVz9Wj7EYP'
base_url = 'https://paper-api.alpaca.markets'

#instantiate REST API
api = tradeapi.REST(api_key, api_secret, base_url, api_version='v2')

def get_data(symbol: str):

    #Single day stock price
    today = date.today()
    current_date = today.strftime("%m-%d-%y")
    NY = 'America/New_York'
    start=pd.Timestamp(current_date, tz=NY).isoformat()
    end=pd.Timestamp(current_date, tz=NY).isoformat()

    api = tradeapi.REST(api_key, api_secret, base_url, api_version='v2')
    bars = api.get_barset([symbol], 'day', start=start, end=end)
    bar_set = bars.df
    bar_set.index.name = "DateTime"
    bar_set.reset_index(inplace=True)
    dataset = pd.DataFrame({'DateTime': bar_set['DateTime'],
                            'Open': bar_set[symbol]['open'],
                            'High': bar_set[symbol]['high'],
                            'Low': bar_set[symbol]['low'],
                            'Close': bar_set[symbol]['close'],
                            'Volume': bar_set[symbol]['volume']})
    return dataset

#obtain account information
account = api.get_account()
print(account)

Account({   'account_blocked': False,
    'account_number': 'PA25EBFUPF1N',
    'buying_power': '387884.446',
    'cash': '88094.74',
    'created_at': '2020-12-10T04:02:05.321554Z',
    'currency': 'USD',
    'daytrade_count': 0,
    'daytrading_buying_power': '387884.446',
    'equity': '100530.24',
    'id': '1a405059-db7e-4563-9723-6ba183ebeb2b',
    'initial_margin': '6217.75',
    'last_equity': '100624.47',
    'last_maintenance_margin': '3125.556',
    'long_market_value': '12435.5',
    'maintenance_margin': '3730.65',
    'multiplier': '4',
    'pattern_day_trader': False,
    'portfolio_value': '100530.24',
    'regt_buying_power': '188624.98',
    'short_market_value': '0',
    'shorting_enabled': True,
    'sma': '0',
    'status': 'ACTIVE',
    'trade_suspended_by_user': False,
    'trading_blocked': False,
    'transfers_blocked': False})


In [4]:
df = get_data('AAPL')
print(df.head())

Empty DataFrame
Columns: [DateTime, Open, High, Low, Close, Volume]
Index: []


In [5]:
s = df['Open']
s

Series([], Name: Open, dtype: object)

In [6]:
# Submit a market order to buy 1 share of Apple at market price
api.submit_order(
    symbol='AAPL',

    qty=1,
    side='buy',
    type='market',
    time_in_force='gtc'
)

Order({   'asset_class': 'us_equity',
    'asset_id': 'b0b6dd9d-8b9b-48a9-ba46-b9d54906e415',
    'canceled_at': None,
    'client_order_id': '40488ad6-34bb-4d42-a598-91ca1bb2fe8e',
    'created_at': '2020-12-24T00:34:12.385442Z',
    'expired_at': None,
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'hwm': None,
    'id': 'ca9f273d-b140-4f54-82fd-699b2a3d00ba',
    'legs': None,
    'limit_price': None,
    'order_class': '',
    'order_type': 'market',
    'qty': '1',
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': 'buy',
    'status': 'accepted',
    'stop_price': None,
    'submitted_at': '2020-12-24T00:34:12.368686Z',
    'symbol': 'AAPL',
    'time_in_force': 'gtc',
    'trail_percent': None,
    'trail_price': None,
    'type': 'market',
    'updated_at': '2020-12-24T00:34:12.385442Z'})

In [7]:
#make this something that takes an input 
ticker= input('Enter ticker: ')

#Create DF of desired company
df = get_data(ticker)

#Isolate stock price
d = df['Open']

amt = input('Amount to be traded: ')
int_amt = int(float(amt) / d[0]) 
stop_price = float(input('Enter stop loss amount: ')) 


# Submit a market order and assign it a Client Order ID.
api.submit_order(
    symbol=ticker,
    #use stock picker input field
    qty=int_amt,
    #take this from amount to be traded on the app 
    #let's say user said $2000/213= 8 stocks you can purschase round(0) Apple stock price it's 213

    side='buy',
    #use signal
    type='market',
    #use market keep it
    time_in_force='day',
    order_class='bracket',
    take_profit=dict(
        limit_price='305.0', 
    ),
    #generate order id
    client_order_id='order5',
    stop_loss=dict(
        stop_price=d[0]*(1-(stop_price/100)),
        #limit_price='100', 
    )

    #for selling at a certain percentage loss of money depends on user input
    #for selling at a certain profit depends on user input will not be used
    #take_profit={'limit_price': symbol_price * 1.05}
)

# Get our order using its Client Order ID.
my_order = api.get_order_by_client_order_id('order5')
print('Got order #{}'.format(my_order.id))


Enter ticker: AAPL
Amount to be traded: 2000
Enter stop loss amount: 5
Got order #acae1824-f97f-4375-8026-e0379702ccee


In [7]:
#Libraries
import pandas_datareader as web
import numpy as np

In [15]:
#Rewriting function for 30 and 100 days
def get_data(symbol: str):

    #Single day stock price
    today = date.today() - timedelta(days=1)
    current_date = today.strftime("%m-%d-%y")
    sdate = date.today() - timedelta(days=149)
    startdate = sdate.strftime("%m-%d-%y")
    NY = 'America/New_York'
    start=pd.Timestamp(startdate, tz=NY).isoformat()
    end=pd.Timestamp(current_date, tz=NY).isoformat()

    api = tradeapi.REST(api_key, api_secret, base_url, api_version='v2')
    bars = api.get_barset([symbol], 'day', start=start, end=end)
    bar_set = bars.df
    bar_set.index.name = "DateTime"
    bar_set.reset_index(inplace=True)
    dataset = pd.DataFrame({'DateTime': bar_set['DateTime'],
                            'Open': bar_set[symbol]['open'],
                            'High': bar_set[symbol]['high'],
                            'Low': bar_set[symbol]['low'],
                            'Close': bar_set[symbol]['close'],
                            'Volume': bar_set[symbol]['volume']})
    return dataset

dataset = get_data('AAPL')
data = dataset['Close']

#Create SMA for 30 days
SMA30 = pd.DataFrame()
SMA30['Close'] = data.rolling(window = 30).mean()

#Create SMA for 100 days
SMA100 = pd.DataFrame()
SMA100['Close'] = data.rolling(window = 100).mean()


#Function for buy and sell
def buy_sell_SMA(data):
    Buy = []
    Sell = []
    flag = -1 #does the price continue to rise or descrease the flag changes when lines cross

    for i in range(0, len(data)):
        if SMA30['Close'][i] > SMA100['Close'][i]:
            if flag != 1:
                Buy.append(data)
                Sell.append(np.nan)
                flag = 1
            else:
                Buy.append(np.nan)
                Sell.append(np.nan)
        elif SMA30['Close'][i] < SMA100['Close'][i]:
            if flag !=0 :
                Buy.append(np.nan)
                Sell.append(data)
            else: 
                Buy.append(np.nan)
                Sell.append(np.nan)
        else:
            Buy.append(np.nan)
            Sell.append(np.nan)


    return (Buy, Sell)


#Create a buy and sell column
a = buy_sell_SMA(data)
data['Buy_Signal_Price'] = a[0]
data['Sell_Signal_Price'] = a[1] 

print(data)

0                                                               373.04
1                                                               380.27
2                                                               384.88
3                                                               425.19
4                                                               435.66
                                           ...                        
102                                                             128.24
103                                                            131.845
104                                                             130.96
Buy_Signal_Price     [nan, nan, nan, nan, nan, nan, nan, nan, nan, ...
Sell_Signal_Price    [nan, nan, nan, nan, nan, nan, nan, nan, nan, ...
Name: Close, Length: 107, dtype: object


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
